# STEP PROJECT

## EDA. Analysis

- EDA : head, tail, shape, category, describe, uniqie ...


Спершу потрібно записати всі осовні імпорти, у тому числі і для пешого етапу - Еди

In [1]:
# Library for machine learning and building graphs
import sklearn as sk
from scipy import stats
from matplotlib import pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer  # to fill missing data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
import logging
import time


import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn")

ImportError: cannot import name 'core' from partially initialized module 'numpy' (most likely due to a circular import) (/home/jovyan/.local/lib/python3.8/site-packages/numpy/__init__.py)

Далі потрібно створити доступ до вже заздалегідь завантаженого на гітхаб ссв файлу та зробити перевірку на статус код

In [ ]:
URL = 'https://raw.githubusercontent.com/axew12/kde_/main/IKEA.csv'
req = requests.get(URL)

In [ ]:
if req.status_code == 200:
    df = pd.read_csv(StringIO(req.text))
else:
    print(f"Failed to fetch data. Status code: {req.status_code}")

In [ ]:
 df.head


In [ ]:
df.tail() # 5 last raws

In [ ]:
df.shape # -> shape

In [ ]:
df.sample(5) # -> 5 randomly selected samples

In [ ]:
df.isnull().sum()

Вище виведений результат показує кількість пустих значень у кожній колонці. Як бачимо, вони наявні у колонках глибини, висоти та ширини виробів, що становлять 39.6% , 26.7%  та 15.9% віж усіх значень відповідно. На мою думку, буде краще заповнити ці нумерік дані за допомогою медіани значення кожної з колонок, адже розміри виборів можуть надалі впливати на коливання ціни, що є важливим для подальшого аналізу.

In [ ]:
req.headers

In [ ]:
df

Далі я вважаю за потрібне проаналізувати колонки та визначитись, які потрібні. Створивши копію дата фрейму, я хотіла б залишити лише такі колонки: 'name', 'category', 'price', 'sellable_online', 'other_colors', 'depth', 'height', 'width', адже саме вони, на мою думку, можуть мати вплив на аналіз даних.

In [ ]:
ikea_df = df.drop(['item_id','Unnamed: 0', 'link', 'old_price', 'short_description'], axis= 1) #1- deleting columns, 0 - raws
ikea_df.columns

In [ ]:
ikea_df.shape

In [ ]:
ikea_df = ikea_df.copy()
ikea_df.dropna(inplace = True )

In [ ]:
ikea_df.shape

Після виділення зайвих колонок та пустих значень, ми отримали майже вдвічі мерший дата врейм. Тепер можна візуалізувати його, аби побачити залежності між змінними.

In [ ]:
# SKALING
from sklearn.preprocessing import MinMaxScaler, StandardScaler
selected_features = ikea_df[['price']]
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(selected_features)
print(scaled_features[:10])

In [ ]:
sns.histplot(scaled_features, kde= True, bins = 25);

In [ ]:
sns.histplot(ikea_df['price'], kde= True, bins = 25);

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np

ikea_df['price'] = pd.to_numeric(ikea_df['price'])
scaler = StandardScaler()
scaled_features_standard = scaler.fit_transform(ikea_df['price'].values.reshape(-1, 1))
print(scaled_features_standard[:10])


In [ ]:
sns.histplot(scaled_features_standard, kde= True, bins = 25, palette='magma');

Також під час роботи виникло припущення щодо важливості масштабування. Питанням, яке ми хотіли дослідити було, чи спростить цей процес аналіз даних по колонці ціна. Як бачимо, результат залишився незмінним, що спростовує гіпотезу. Для того, аби зробити стандартне масштабуванндля фітчу прайс, ми перевели колонку з одновимірного у двовимірний вигляд за допомогою рішейпу. <br>
Також можна перевірити це ж питання на колонці одного із параметрів виробів. Я оберу висоту 

In [ ]:
# SKALING
from sklearn.preprocessing import MinMaxScaler, StandardScaler
selected_features = ikea_df[['height']]
scaler = MinMaxScaler()
scaled_features_h = scaler.fit_transform(selected_features)
print(scaled_features[:10])

In [ ]:
sns.histplot(ikea_df['height'], kde= True, bins = 25);

In [ ]:
sns.histplot(scaled_features_h, kde= True, bins = 25);

Як бачимо, гіпотеза теж спростовується.<br>
Далі мало б сенс вивести теплову карту, яка б дозволила проаналізувати кореляцію одразу багатьох змінних. Найбільше нас цікавлять розміри та ціна обʼєктів.

In [ ]:
sns.heatmap(ikea_df[['price', 'depth', 'height', 'width']].corr(), xticklabels= df[['price', 'depth', 'height', 'width']].corr().columns,
            yticklabels= ikea_df[['price', 'depth', 'height', 'width']].corr().columns, center= 0, annot= True);

In [ ]:
sns.heatmap(ikea_df.corr(method= 'spearman'), xticklabels= ikea_df.corr().columns,
            yticklabels= ikea_df.corr().columns, center= 0, annot= True);

Найкращу кореляцію бачимо між зміними ціни та ширини (77%) -> що дає можливість припускати, що найбільше ціна залежить від ширини виробу. Також непоганий рівень кореляції між ціною та глибиною виробів (62%). Тобто можна припустити, що розміри виробів усе ж впливають на ціну.<br>
Теплова карта Спермана дає більш чіткі цифри для аналізу, хоч трохи змінюючи результат. Топ дві кореляції залишаються незмінними, проте їхній рівень та різниця між ними дещо змінюється: ціна * ширина (79%) та ціна * глибина (47%). Також додається ще одна кореляція між ціною та глибиною (47%), що ще більше підтверджує гіпотезу того, що ціна все ж залежить від розмірів

In [ ]:
sns.pairplot(ikea_df[[ 'price', 'depth', 'height', 'width', 'other_colors', 'category']], hue = 'other_colors')

In [ ]:
sns.boxplot(data=ikea_df, x='other_colors', y='price')

In [ ]:
plt.subplots(figsize = (12,10))
sns.boxplot(data = df, x = 'price', y = 'category', hue = 'other_colors')
plt.show()

Проаналізувавши боксплот вище, ми бачимо, що часто в категорії вироби без кольорів коштують більше від інших, проте у тих, які мають палітру кольорів, є багато аутлаєрів, через що середня вартість може бути спотвореною чи зміненою.

In [ ]:
plt.subplots(figsize = (12,10))
sns.scatterplot(x = 'price', y = 'width', data = ikea_df, hue = 'other_colors', palette='inferno');
plt.grid()

In [ ]:
plt.subplots(figsize = (12,10))
sns.scatterplot(x = 'price', y = 'width', data = ikea_df, hue = 'category', palette='inferno');
plt.grid()

Метою побудови графіків вище було визначити,чи є звʼязок між ціною та наявністю інших кольорів виробу. Ми бачимо, що загалом виробів із лише одним кольором менше, аніж із багатьма. Проте боксплот та скетерплот показали, що ціна виробів із палітрою кольорів дещо вища, хоч різниця і не суттєва. Також у скетерплотах на осі у я зобразила ширину - найбільш корельовану із ціною змінну, й можна припустити, що зі збільшенням ширини , а тобто і розмірів , ціна підвищувалась. На останньому графіку я вирішила зобразити той самий скетерплот, розбивши кольори за категоріями, але якоїсь супер чіткої картини я не побачила. Найдорожчими вивявились дивани й крісла, які також є одними із найбільших за  шириною виробами.

In [ ]:
plt.subplots(figsize = (10,8))
sns.countplot( palette='magma', x = df['category']).set_xticklabels(df['category'].unique(), rotation = 80)
plt.show()

## Machine Learning

Першим кроком можна перевірити точність наших моделей.

In [ ]:
ikea_df['designer'].head(15)

In [ ]:
# Create function to clear Designers
def cleanDesigners(value, removeIKEA=False, emptyValue=np.nan):
    if not isinstance(value, str):
        return value

    if len(value)>0 and value[0].isdigit():
        return emptyValue

    designers = value.split("/")

    if removeIKEA:
        try:
            designers.remove("IKEA of Sweden")
        except:
            pass
    if len(designers) > 0:
        return '/'.join(sorted(designers))
    else:
        return emptyValue

In [ ]:
ikea_df = pd.read_csv("https://raw.githubusercontent.com/axew12/kde_/main/IKEA.csv", index_col= 0).drop_duplicates()

ikea_df['designer_clean'] = df['designer'].apply(cleanDesigners, args= (False, "IKEA of Sweden"))

In [ ]:
ikea_df['designer_clean'].head()

In [ ]:
X = ikea_df[['depth', 'width', 'height', 'category', 'designer_clean', 'other_colors']]
Y = ikea_df['price']
X_train, X_test, Y_train, Y_test = sk.model_selection.train_test_split(X, Y, test_size=0.2, random_state=42)

numeric_transf = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('impute', SimpleImputer(strategy='median'))
])


categorical_transf = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


col_prepr = ColumnTransformer(transformers=[
    ('numeric', numeric_transf, ['depth', 'width', 'height']),
    ('categorical', categorical_transf, ['category', 'designer_clean', 'other_colors'])
])

dtr = Pipeline(steps=[
    ('col_prep', col_prepr),
    ('dtr', DecisionTreeRegressor(max_depth=10, random_state=42))
])

dtr.fit(X_train, Y_train)
dtr_predict = dtr.predict(X_test)
# Use different metric for out model
print('R^2 : {:.5f}'.format(dtr.score(X_test, Y_test)))
print('MAE : {:.5f}'.format(sk.metrics.mean_absolute_error(dtr_predict, Y_test)))
print('MSE : {:.5f}'.format(np.sqrt(sk.metrics.mean_squared_error(dtr_predict, Y_test))))

In [ ]:
X_train.shape, X_test.shape

In [3]:
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier|
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, train_test_split

SyntaxError: invalid syntax (<ipython-input-3-f32fe89ec59c>, line 6)

In [ ]:
import logging 
import time 
logging.basicConfig(filename = 'machine_learning_traning.log', filemode= 'w', level= logging.INFO, format= '%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
def start_pipeline(pipeline, X_train, Y_train, X_test, Y_test) -> None: 
    start_time = time.time()
    logging.info('Start model traning')
    pipeline.fit(X_train, Y_train)
    dtr_predict = pipeline.predict(X_test)
    logging.info(f'Model: {pipeline} Accuracy: {str(pipeline.score(X_test, Y_test))}' )
    

In [ ]:
start_pipeline(dtr, X_train, Y_train, X_test, Y_test)

In [ ]:
dtr

In [ ]:
# Fill in data 
ikea_df['other_colors_1'] = ikea_df['other_colors'].map(dict(Yes=1, No=0))

median_d = ikea_df.groupby(['category'])['depth'].median()
median_h = ikea_df.groupby(['category'])['height'].median()
median_w = ikea_df.groupby(['category'])['width'].median()

median_price = ikea_df.groupby(['category'])['price'].median()
median_dsgn =  ikea_df.groupby(['designer_clean'])['price'].median()

ikea_df = ikea_df.set_index(['category'])
ikea_df['depth_1'] = ikea_df['depth'].fillna(median_d)
ikea_df['height_1'] = ikea_df['height'].fillna(median_h)
ikea_df['width_1'] = ikea_df['width'].fillna(median_w)
ikea_df['category_median_price'] = median_price

ikea_df = ikea_df.reset_index()

ikea_df = ikea_df.set_index(['designer_clean'])
ikea_df['designer_median_price'] = median_dsgn
ikea_df = ikea_df.reset_index()
ikea_df.head()

In [ ]:
# Function to choose BestRegressor
# Step by step fit data for different model, and display result
def getBestRegressor (X, Y):
    X_train, X_test, Y_train, Y_test = sk.model_selection.train_test_split(X, Y, test_size=0.2, random_state=42)
    models = [
        sk.linear_model.LinearRegression(),
        sk.linear_model.LassoCV(),
        sk.linear_model.RidgeCV(),
        sk.svm.SVR(kernel='linear'),
        sk.neighbors.KNeighborsRegressor(n_neighbors=16),
        sk.tree.DecisionTreeRegressor(max_depth=10, random_state=42),
        RandomForestRegressor(random_state=42),
        GradientBoostingRegressor()
    ]
    
    TestModels = pd.DataFrame()
    res = {}
    tmp = {}
    
    for model in models:
        m = str(model)
        tmp['Model'] = m[:m.index('(')]
        model.fit(X_train, Y_train)
        tmp['R^2'] = '{:.5f}'.format(model.score(X_test, Y_test))
        tmp['MAE'] = '{:.5f}'.format(sk.metrics.mean_absolute_error(model.predict(X_test), Y_test))
        tmp['RMSE'] = '{:.5f}'.format(np.sqrt(sk.metrics.mean_squared_error(model.predict(X_test), Y_test)))
        
        TestModels = pd.concat([TestModels, pd.DataFrame([tmp])])
        
    TestModels.set_index('Model', inplace=True)
    res['model'] = TestModels
    res['X_train'] = X_train
    res['Y_train'] = Y_train
    res['X_test'] = X_test
    res['Y_test'] = Y_test
    return res


In [ ]:
X1 = ikea_df[['width_1','depth_1','height_1']]
Y1 = ikea_df['price']

test1 = getBestRegressor(X1, Y1)
test1['model'].sort_values(by='R^2', ascending=False)

In [ ]:
numik = ikea_df[['depth_1', 'width_1', 'height_1','price']]
numik.head

In [ ]:
X = ikea_df[['depth_1', 'width_1', 'height_1']]
Y = ikea_df['price']
x_train, x_test, y_train, y_test = sk.model_selection.train_test_split(X, Y, test_size=0.2, random_state=42)

numeric_transf = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('impute', SimpleImputer(strategy='median'))
])




col_prepr = ColumnTransformer(transformers=[
    ('numeric', numeric_transf, ['depth_1', 'width_1', 'height_1'])

])

dtr = Pipeline(steps=[
    ('col_prep', col_prepr),
    ('dtr', DecisionTreeRegressor(max_depth=10, random_state=42))
])

dtr.fit(x_train, y_train)
#dtr_predict = dtr1.predict(x_test)
# Use different metric for out model
print('R^2 : {:.5f}'.format(dtr.score(x_test, y_test)))
print('MAE : {:.5f}'.format(sk.metrics.mean_absolute_error(dtr_predict, y_test)))
print('MSE : {:.5f}'.format(np.sqrt(sk.metrics.mean_squared_error(dtr_predict, y_test))))

In [ ]:
def float_to_int(float_list):
    int_list = []
    for float_num in float_list:
        int_num = round(float_num)
        int_list.append(int_num)
    return int_list

In [ ]:
y_test_int = float_to_int(y_test)
y_train_int = float_to_int(y_train)

In [ ]:
tree = DecisionTreeClassifier(max_depth = 5, random_state = 0)
tree.fit(x_train, y_train_int )
print("Правильность на обучающем наборе: {:.3f}".format(tree.score(x_train, y_train_int )))
print("Правильность на тестовом наборе: {:.3f}".format(tree.score(x_test, y_test_int)))

In [ ]:
dt_param_grid = {'max_depth': [2,3, 4, 5, 7, None]}
knn_param_grid = {'n_neighbors': [i for i in range(1, 16)]}

dt_gs = GridSearchCV(DecisionTreeClassifier(), dt_param_grid, cv=5)
knn_gs = GridSearchCV(KNeighborsClassifier(), knn_param_grid, cv=5)


dt_gs.fit(x_train, y_train_int)
knn_gs.fit(x_train, y_train_int)

print(f"Decision Tree: best parameters {dt_gs.best_params_}, best score {dt_gs.best_score_}")
print(f"KNN: best parameters {knn_gs.best_params_}, best score {knn_gs.best_score_}")